In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

Будем решать задачу на определение дату следующего ТО и пробега на эту дату по некоторым характеристикам автомобилей. Для начала загрузим обучающие и тестовые данные.

In [2]:
train_data_raw = pd.read_csv('train.csv', delimiter=';', index_col=0)
test_data_raw = pd.read_csv('test.csv', delimiter=';', index_col=0)

## Первичная обработка

Объединим их на время обработки в одну таблицу, чтобы модифицировать их одновременно. После сможем просто отделить их по значению is_train.

In [3]:
train_data_raw['is_train'] = True
test_data_raw['is_train'] = False

data = pd.concat([train_data_raw, test_data_raw], ignore_index=True)

Переименуем названия колонок, чтобы было проще и быстрее к ним обращаться.

In [4]:
data.columns = ['model', 'brand', 'vin', 'visit_mileage', 'year', 'visit_date', 'service_interval', 'is_service', 'is_train']

Посмотрим некоторую информацию таблицы и вообще что она из себя представляет.

In [5]:
data

,model,brand,vin,visit_mileage,year,visit_date,service_interval,is_service,is_train
0,LADA Largus Caravan (7м),LADA,27837,29 644,01.01.2014 0:00:00,03.08.2015 0:00:00,15 000,Нет,True
1,LADA Largus Caravan (7м),LADA,27837,29 644,01.01.2014 0:00:00,06.08.2015 0:00:00,15 000,Да,True
2,LADA Largus Caravan (7м),LADA,27837,62 152,01.01.2014 0:00:00,28.04.2016 0:00:00,15 000,Нет,True
3,LADA Largus Caravan (7м),LADA,27837,104 743,01.01.2014 0:00:00,09.12.2016 0:00:00,15 000,Да,True
4,УАЗ Patriot 3163,УАЗ,28253,29 979,01.01.2016 0:00:00,27.01.2018 0:00:00,15 000,Да,True
...,...,...,...,...,...,...,...,...,...
119709,Toyota RAV 4 IV 5D,TOYOTA,8383,33 521,01.01.2015 0:00:00,17.10.2018 0:00:00,10 000,Да,False
119710,Hyundai Solaris Sedan,Hyundai,39104,25 552,01.01.2014 0:00:00,18.11.2018 0:00:00,15 000,Да,False
119711,Lexus NX 200T,LEXUS,5059,30 099,01.01.2016 0:00:00,01.04.2019 0:00:00,10 000,Да,False
119712,Toyota Land Cruiser 200,TOYOTA,7698,189 650,01.01.2013 0:00:00,15.02.2019 0:00:00,10 000,Да,False


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119714 entries, 0 to 119713
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   model             119714 non-null  object
 1   brand             119714 non-null  object
 2   vin               119714 non-null  int64 
 3   visit_mileage     119310 non-null  object
 4   year              119714 non-null  object
 5   visit_date        119714 non-null  object
 6   service_interval  119714 non-null  object
 7   is_service        119714 non-null  object
 8   is_train          119714 non-null  bool  
dtypes: bool(1), int64(1), object(7)
memory usage: 7.4+ MB


Как видно, почти все признаки представлены как object. Давайте преобразуем их в соответствующие форматы. Сперва преобразуем категориальные признаки.

In [7]:
categorical_cols = ['model', 'brand', 'is_service']
data[categorical_cols] = data[categorical_cols].astype('category')
data['is_service'] = data['is_service'].map({'Да': 1, 'Нет': 0})

Далее с датами. У признака year, думаю, целесообразно оставить только год.

In [8]:
data['visit_date'] = pd.to_datetime(data['visit_date'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
data['year'] = pd.to_datetime(data['year'], format='%d.%m.%Y %H:%M:%S', errors='coerce').dt.year

Числовые признаки представлены как строка числа, разделённого пробелами по разрядам. Уберём их и преобразуем в числа.

In [9]:
data['visit_mileage'] = data['visit_mileage'].str.replace(r'\s+', '', regex=True)
data['visit_mileage'] = pd.to_numeric(data['visit_mileage'])

data['service_interval'] = data['service_interval'].str.replace(r'\s+', '', regex=True)
data['service_interval'] = pd.to_numeric(data['service_interval'])

Посмотрим есть ли пропущенные значения.

In [10]:
data.isna().sum()

model                 0
brand                 0
vin                   0
visit_mileage       404
year                250
visit_date            0
service_interval      0
is_service            0
is_train              0
dtype: int64

В столбцах visit_mileage и year есть пропущенные значения. Так как их мало по сравнению с размерами таблицы, можно просто убрать строки с ними.

In [11]:
data = data.dropna()

После этого можно преобразовать year из float в int, так как пропцщенные значения этому более не мешают.

In [12]:
data['year'] = data['year'].astype('int64')

C:\Users\kento\AppData\Local\Temp\ipykernel_11304\604412865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'] = data['year'].astype('int64')


Наконец посмотрим на результаты изменений.

In [13]:
data

,model,brand,vin,visit_mileage,year,visit_date,service_interval,is_service,is_train
0,LADA Largus Caravan (7м),LADA,27837,29644.0,2014,2015-08-03,15000,0,True
1,LADA Largus Caravan (7м),LADA,27837,29644.0,2014,2015-08-06,15000,1,True
2,LADA Largus Caravan (7м),LADA,27837,62152.0,2014,2016-04-28,15000,0,True
3,LADA Largus Caravan (7м),LADA,27837,104743.0,2014,2016-12-09,15000,1,True
4,УАЗ Patriot 3163,УАЗ,28253,29979.0,2016,2018-01-27,15000,1,True
...,...,...,...,...,...,...,...,...,...
119709,Toyota RAV 4 IV 5D,TOYOTA,8383,33521.0,2015,2018-10-17,10000,1,False
119710,Hyundai Solaris Sedan,Hyundai,39104,25552.0,2014,2018-11-18,15000,1,False
119711,Lexus NX 200T,LEXUS,5059,30099.0,2016,2019-04-01,10000,1,False
119712,Toyota Land Cruiser 200,TOYOTA,7698,189650.0,2013,2019-02-15,10000,1,False


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119060 entries, 0 to 119713
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   model             119060 non-null  category      
 1   brand             119060 non-null  category      
 2   vin               119060 non-null  int64         
 3   visit_mileage     119060 non-null  float64       
 4   year              119060 non-null  int64         
 5   visit_date        119060 non-null  datetime64[ns]
 6   service_interval  119060 non-null  int64         
 7   is_service        119060 non-null  category      
 8   is_train          119060 non-null  bool          
dtypes: bool(1), category(3), datetime64[ns](1), float64(1), int64(3)
memory usage: 6.0 MB


С этим уже удобнее работать и можно приступать к следующим этапам обработки. Под конец разделим на таблицу как было в начале.

In [15]:
train_data = data[data['is_train'] == True].drop(columns=['is_train'])
test_data = data[data['is_train'] == False].drop(columns=['is_train'])

## Создание новых признаков

Так как нам надо предсказывать дату следующего ТО, то целесообразно будет убрать записи, которые не являются посещениями по ТО.

In [16]:
train_data = train_data[train_data['is_service'] == 1].drop(columns=['is_service'])

Добавим признаки: среднее время между визитами и средний пробег между визитами. Они будут для каждой уникальной машины (по VIN-номеру). Для этого сначала для каждой записи вычислим время и пробег с последнего визита.

In [26]:
train_data = train_data.sort_values(['vin', 'visit_date'])
train_data['days_since_service']=train_data.groupby('vin')['visit_date'].diff().dt.days
train_data['mileage_since_service'] = train_data.groupby('vin')['visit_mileage'].diff()

Для записей, у которых указана дата первого визита по ТО, сейчас этих неопределены. Заполним их нулями.

In [28]:
train_data['days_since_service'] = train_data['days_since_service'].fillna(0)
train_data['mileage_since_service'] = train_data['mileage_since_service'].fillna(0)

Перейдем к средним значениям

In [29]:
vin_stats = train_data.groupby('vin').agg({
    'days_since_service': 'mean',
    'mileage_since_service': 'mean',
}).rename(columns={
    'days_since_service': 'avg_days',
    'mileage_since_service': 'avg_mileage',
}).reset_index()

Объеденяем

In [31]:
train_data = train_data.merge(vin_stats, on='vin', how='left')

Удалим столбцы со днями и пробегом с последнего визита. Вообще они могли бы помочь в тренировке модели, но их нет в тестовой таблице.

In [33]:
train_data = train_data.drop(columns=['days_since_service', 'mileage_since_service'])

## Обработка категориальных признаков

Имеем два категориальных признака: модель и бренд автомобиля. Разберемся сначала с брендом, посмотрим сколько там уникальных значений.

In [44]:
train_data.brand.unique()

['TOYOTA', 'LEXUS', 'MERCEDES-BENZ', 'MITSUBISHI', 'INFINITI', ..., 'CHEVROLET NIVA', 'ISUZU', 'УАЗ', 'GENESIS', 'KIA']
Length: 20
Categories (20, object): ['BMW', 'CHEVROLET NIVA', 'Citroen', 'FORD', ..., 'Skoda', 'TOYOTA', 'ГАЗ', 'УАЗ']

Всего 20 уникальных значений. Это немного, поэтому будем использовать метод One-Hot Encoding.

In [45]:
train_data = pd.get_dummies(train_data, columns=['brand'])

Посмотрим признак model

In [47]:
train_data.model.unique()

['Toyota Estima', 'Lexus RX III 350', 'Lexus RX II 330', 'Mercedes-Benz GL-klasse X164', 'Mercedes-Benz M-klasse AMG W166', ..., 'KIA Rio IV', 'Hyundai Solaris Sedan', 'Hyundai Solaris Hatchback', 'Hyundai Creta', 'Hyundai Solaris HCR']
Length: 411
Categories (411, object): ['172411', '38787-0000010-51', '38787-0000010-94', '38787-92', ..., 'УАЗ-315148', 'УАЗ-390945', 'УАЗ-390995', 'УАЗ-39623-СГР']

Их достаточно много, поэтому используем метод Label Encoding

In [48]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_data['model'] = le.fit_transform(train_data['model'])

## Масштабирование количественных признаков